In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

In [ ]:



# If required, create a face detection pipeline using MTCNN:
mtcnn = MTCNN(image_size=320, margin=150) # TO RESEARCH, if what will be better bbox with face or whole photo

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
dir_ = "lfw"
photo_paths = glob.glob(dir_+"/*/*")

In [ ]:
labels = []
for item in photo_paths:
    name_path = os.path.dirname(item)
    name = os.path.basename(name_path)
    name
    labels.append(name)

In [ ]:
df = pd.DataFrame(dict(label=labels, photo_paths=photo_paths))
print(df.shape)
df.head()

In [ ]:
def chunks(li, chunk_size=100):
    iters = (len(li)/chunk_size)
    for i in range(int(iters)):
        i = i*chunk_size
        yield li[i:i+chunk_size]

In [ ]:
i = 0

embeddings = []
for item in df.photo_paths:
    img = Image.open(item)
    img_cropped = mtcnn(img)
    del img
    # # Calculate embedding (unsqueeze to add batch dimension)
    img_embedding = resnet(img_cropped.unsqueeze(0))
    del img_cropped
    embeddings.append(img_embedding.detach().numpy())
    del img_embedding


pd.Series(embeddings).to_csv(f"tmp/file_{i}.csv")
i += 1
    
    

In [ ]:
df["img_embedding"] = embeddings

In [ ]:
results = []
for i in range(len(labels)-1):
    if labels[i] == labels[i+1]:
        results.append(1)
    else:
        results.append(0)


In [ ]:
pd.Series(results).value_counts()

In [ ]:
df["img_embedding2"] = df.img_embedding.shift(-1)

In [ ]:
df = df.dropna()

In [ ]:
df["y"] = results

In [ ]:
df["x"] = df.apply(lambda x: x.img_embedding2-x.img_embedding, axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

prepare data for lr input

In [ ]:
df["x"] = df.x.apply(lambda x: x[0])

In [ ]:
x = df.x.values
x = np.stack(x)

In [ ]:
lr = LogisticRegression(penalty="l2", C=0.000001)
lr.fit(np.stack(df.x.values), df.y.values)

In [ ]:
pred = lr.predict(np.stack(df.x.values))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
precision_recall_fscore_support(df.y, pred, average="binary")

In [ ]:
pd.Series(pred).value_counts()

In [ ]:
import optuna

In [ ]:
def objective(trial):
    lr = LogisticRegression(
        penalty="elasticnet",
        solver="saga",
        C=trial.suggest_float('C', 1e-10, 1e10, log=True),
        l1_ratio=trial.suggest_float('l1_ratio', 0, 1))
    lr.fit(np.stack(df.x.values), df.y.values)
    pred = lr.predict(np.stack(df.x.values))
    prec, recc, f1, support = precision_recall_fscore_support(df.y, pred, average="binary")

    return f1  # An objective value linked with the Trial object.

study = optuna.create_study()  # Create a new study.
study.optimize(objective, n_trials=100) 